### 1. Transformer Architecture



attention 계산식을 생각하며 빈칸을 채워봅시다!!

In [1]:
import math
import random
from typing import Tuple

import torch
import torch.nn as nn


# -------------------------
# A. Scaled Dot-Product Attention
# -------------------------
class ScaledDotProductAttention(nn.Module):
    def __init__(self, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(dropout)

    def forward(self, Q, K, V, mask=None):
        """
        Q,K,V: (batch, heads, seq_len, d_k)
        mask:  (batch, 1 or heads, seq_len, seq_len)
        """
        d_k = Q.size(-1)
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(d_k)  # (B,H,L,L)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, float('-inf'))
        attn = torch.softmax(scores, dim=-1)
        attn = self.dropout(attn)
        output = torch.matmul(attn, V)  # (B,H,L,d_k)
        return output, attn


문제 1) 아래 코드를 살펴보고, 단순 Attention 대신 Multi-Head Attention을 사용하는 이유를 설명하시오.

- Multi-Head Attention은 여러개의 어텐션 헤드가 병렬적으로 정보를 처리한다. 이런 처리 방식은 모델이 문맥을 파악할 수 있게 하여 표현력이 향상된다.

문제 2) Positional Encoding의 기능에 대해 설명하시오.

- 단어의 위치 정보를 모델에 알려준다. 문장 내 각 단어의 위치를 나타내는 정보를 단어 임베딩 벡터에 더하여 모델이 단어 순서를 이해할 수 있도록 돕는다.


In [2]:
# -------------------------
# B. Multi-Head Attention
# -------------------------
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads, dropout=0.1):
        super().__init__()
        assert d_model % num_heads == 0
        self.d_k = d_model // num_heads
        self.num_heads = num_heads

        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        self.layernorm = nn.LayerNorm(d_model)

        self.attn = ScaledDotProductAttention(dropout=dropout)

    def _split_heads(self, x):
        B, L, D = x.shape
        x = x.view(B, L, self.num_heads, self.d_k).transpose(1, 2)
        return x

    def _combine_heads(self, x):
        B, H, L, d_k = x.shape
        x = x.transpose(1, 2).contiguous().view(B, L, H * d_k)
        return x

    def forward(self, x, kv=None, mask=None):
        residual = x
        if kv is None:
            kv = x
        Q = self._split_heads(self.W_q(x))
        K = self._split_heads(self.W_k(kv))
        V = self._split_heads(self.W_v(kv))
        ctx, _ = self.attn(Q, K, V, mask=mask)          # (B,H,L_q,d_k)
        out = self._combine_heads(ctx)                  # (B,L_q,D)
        out = self.dropout(self.W_o(out))
        return self.layernorm(out + residual)

# -------------------------
# C. Positional Encoding (sin/cos)
# -------------------------
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-(math.log(10000.0) / d_model)))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # (1,L,D)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1), :].to(x.dtype)
        return self.dropout(x)

# -------------------------
# D. Position-wise FFN
# -------------------------
class PositionwiseFFN(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_ff, d_model),
            nn.Dropout(dropout),
        )
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x):
        residual = x
        x = self.net(x)
        return self.norm(x + residual)

Transformer의 인코더와 디코더 레이어 구조를 생각하며 빈칸을 채워봅시다!!

In [3]:
# -------------------------
# E. Encoder
# -------------------------
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads, dropout)
        self.ffn = PositionwiseFFN(d_model, d_ff, dropout)

    def forward(self, x, src_mask=None):
        x = self.self_attn(x, kv=None, mask=src_mask)
        x = self.ffn(x)
        return x

class Encoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, num_heads, d_ff, dropout=0.1, max_len=5000):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, d_model)
        self.posenc = PositionalEncoding(d_model, max_len, dropout)
        self.layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(N)])

    def forward(self, src, src_mask=None):
        x = self.embed(src) * math.sqrt(d_model := self.embed.embedding_dim)
        x = self.posenc(x)
        for layer in self.layers:
            x = layer(x, src_mask=src_mask)
        return x

# -------------------------
# F. Decoder
# -------------------------
def generate_subsequent_mask(sz: int):
    mask = torch.tril(torch.ones(sz, sz)).bool()
    return mask.unsqueeze(0).unsqueeze(0)

class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads, dropout)
        self.cross_attn = MultiHeadAttention(d_model, num_heads, dropout)
        self.ffn = PositionwiseFFN(d_model, d_ff, dropout)

    def forward(self, x, enc_out, tgt_mask=None, memory_mask=None):
        x = self.self_attn(x, kv=None, mask=tgt_mask)
        x = self.cross_attn(x, kv=enc_out, mask=memory_mask)
        x = self.ffn(x)
        return x

class Decoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, num_heads, d_ff, dropout=0.1, max_len=5000):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, d_model)
        self.posenc = PositionalEncoding(d_model, max_len, dropout)
        self.layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(N)])
        self.norm = nn.LayerNorm(d_model)

    def forward(self, tgt, enc_out, tgt_mask=None, memory_mask=None):
        x = self.embed(tgt) * math.sqrt(d_model := self.embed.embedding_dim)
        x = self.posenc(x)
        for layer in self.layers:
            x = layer(x, enc_out, tgt_mask=tgt_mask, memory_mask=memory_mask)
        return self.norm(x)

# -------------------------
# G. 전체 Transformer + 마스크
# -------------------------
class Transformer(nn.Module):
    def __init__(self, src_vocab, tgt_vocab, d_model=256, N=4, heads=4, d_ff=1024, dropout=0.1, max_len=512):
        super().__init__()
        self.encoder = Encoder(src_vocab, d_model, N, heads, d_ff, dropout, max_len)
        self.decoder = Decoder(tgt_vocab, d_model, N, heads, d_ff, dropout, max_len)
        self.generator = nn.Linear(d_model, tgt_vocab)

    def make_src_mask(self, src):
        return (src != PAD).unsqueeze(1).unsqueeze(1)  # (B,1,1,Ls)

    def make_tgt_mask(self, tgt):
        B, L = tgt.shape
        pad = (tgt != PAD).unsqueeze(1).unsqueeze(1)   # (B,1,1,Lt)
        causal = generate_subsequent_mask(L).to(tgt.device)  # (1,1,Lt,Lt)
        return pad & causal

    def forward(self, src, tgt):
        src_mask = self.make_src_mask(src)
        tgt_mask = self.make_tgt_mask(tgt)
        memory = self.encoder(src, src_mask=src_mask)
        out = self.decoder(tgt, memory, tgt_mask=tgt_mask, memory_mask=src_mask)
        logits = self.generator(out)
        return logits



### 2. Self-Supervised Learning

#### 문제 1) Autoencoding Language Model
아래 세 문장에서 BERT가 [MASK] 위치에 대해 예측한 1순위 토큰이 문맥상 적절한지 평가하세요.

적절하다면, 왜 해당 토큰이 자연스럽다고 볼 수 있는지 근거를 제시하세요.

적절하지 않다면, 그 이유가 문맥 이해 부족 때문인지, 아니면 훈련 데이터 분포(자주 등장하는 표현) 때문인지 분석해 보세요.

1. 'something'은 적절하다. wondering과 문맥적으로 잘 어울리는 단어.
2. 'beach'는 적절하다. 친구와 주말에 갈 수 있는 장소로 적절함.
3. 'home'은 적절하다. 집에 우산을 두고온 일반적인 상황 표현을 잘 학습함.

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM

mlm_name = "distilbert-base-uncased"  # 경량 BERT
tok = AutoTokenizer.from_pretrained(mlm_name)
model = AutoModelForMaskedLM.from_pretrained(mlm_name)
model.eval()

def topk_mask_fill(text, k=5):
    inputs = tok(text, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    mask_idx = (inputs.input_ids[0] == tok.mask_token_id).nonzero(as_tuple=True)[0].item()
    probs = torch.softmax(logits[0, mask_idx], dim=-1)
    topk_ids = torch.topk(probs, k=k).indices.tolist()
    return [(tok.decode([i]), float(probs[i])) for i in topk_ids]

sentences = [
    "I'm wondering if I should eat [MASK] for lunch today.",
    "I decided to go to the [MASK] with my friends this weekend.",
    "It started to rain and I remembered I left my umbrella at [MASK]."
]

for s in sentences:
    print("\nInput:", s)
    preds = topk_mask_fill(s, k=5) # top-k 자유롭게 수정 가능
    for t,p in preds:
        print(f"  - {t:15s}  p={p:.4f}")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



Input: I'm wondering if I should eat [MASK] for lunch today.
  - something        p=0.0690
  - here             p=0.0688
  - breakfast        p=0.0414
  - dinner           p=0.0405
  - pizza            p=0.0374

Input: I decided to go to the [MASK] with my friends this weekend.
  - beach            p=0.1086
  - movies           p=0.0727
  - gym              p=0.0478
  - mall             p=0.0325
  - zoo              p=0.0305

Input: It started to rain and I remembered I left my umbrella at [MASK].
  - home             p=0.0831
  - night            p=0.0602
  - school           p=0.0366
  - dawn             p=0.0308
  - lunch            p=0.0271


### 3. Prompt Engineering

아래는 동일한 질문에 대해 Baseline Prompt와 Engineered Prompt를 사용했을 때의 모델 답변이다.
두 결과를 비교하고, 왜 프롬프트 엔지니어링(prompt engineering)이 중요한지 서술하시오.

또한, 프롬프트 엔지니어링 기법에 대해 설명하시오.

Baseline Prompt에 대하여 모델은 **5th, 2024** 라는 결과를 출력하였다. 이는 baseline의 모든 정보를 반영하지 않고 있다. Engineered prompt는 정확하게 **5/05/2024** 라는 결과를 출력한다. 모델이 훨씬 사용자의 의도를 반영한 결과를 출력할 수 있었다.
의도가 명확해지고, 정확성이 향상되는 결과를 도출할 수 있다.

프롬프트 엔지니어링이란 모델이 특정 작업을 수행하도록 명확한 명령을 내리거나(명령 기반 프롬프트), 특정 역할을 부여하여 답변을 생성하도록 유도하는 방식(역할 기반 프롬프트) 등이 존재한다.

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# -----------------------------
# 1) 모델 로드
# -----------------------------
model_id = "google/flan-t5-base"
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id).to(device)

def generate(prompt, max_new_tokens=128):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# -----------------------------
# 2) 예제 프롬프트들 (Baseline vs Engineered)
# -----------------------------
prompts = {
    "convert date": {
        "baseline": '''Convert March 5th, 2024 to YYYY-MM-DD format ''',

        "engineered": '''You are a date parser.
Task: Convert the input into exactly YYYY-MM-DD format (4-digit year, 2-digit month, 2-digit day).
Rules:
- Output ONLY the date in that format.
- No extra text or explanation.
Input: "March 5th, 2024"
Output:'''
    }

    }


# -----------------------------
# 3) 실행 및 비교 출력
# -----------------------------
for task, variants in prompts.items():
    print("="*80)
    print(f"📝 Task: {task}")

    for kind, prompt in variants.items():
        output = generate(prompt)
        print(f"\n--- {kind.upper()} Prompt ---")
        print(prompt)
        print(f"\n👉 Model Output:\n{output}\n")


📝 Task: convert date

--- BASELINE Prompt ---
Convert March 5th, 2024 to YYYY-MM-DD format 

👉 Model Output:
5th, 2024


--- ENGINEERED Prompt ---
You are a date parser.
Task: Convert the input into exactly YYYY-MM-DD format (4-digit year, 2-digit month, 2-digit day).
Rules:
- Output ONLY the date in that format.
- No extra text or explanation.
Input: "March 5th, 2024"
Output:

👉 Model Output:
"5/05/2024"



### 4.RAG


In [7]:
!pip install -qU langchain langchain_community sentence-transformers faiss-cpu transformers accelerate langchain-core langchain-upstage bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [15]:
pip install --upgrade langchain transformers sentence-transformers torch

  Using cached transformers-4.56.1-py3-none-any.whl.metadata (42 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 80.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 76.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.3
    Uninstalling transformers-4.46.3:
      Successfully uninstalled transformers-4.46.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-upstage 0.7.3 requires tokenizers<0.21.0,>=0.20.0, but you have tokenizers 0.22.0 which is incompatible.


#### 문제1) 아래 코드를 참고하여 RAG 프로세스를 서술해주세요

#### 문제2) sample.md 를 업로드해서 아래 샘플 질문들을 입력해 결과를 출력해보세요.

1. "인공지능의 역사에서 튜링 테스트란 무엇인가요?"
2. "딥러닝 혁명은 언제 시작되었나요?"
3. "데이터 분석에 대해 설명하세요"


In [6]:
import os, tempfile
from google.colab import files
from langchain_upstage import ChatUpstage
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA

# 파일 업로드 함수
def upload_file():
    print("문서 파일을 업로드해주세요.")
    uploaded = files.upload()
    file_path = list(uploaded.keys())[0]
    print(f"업로드 완료: {file_path}")
    return file_path

# -----------------------------
# 1) Solar-Pro2 LLM 로드
# -----------------------------
chat = ChatUpstage(
    api_key="up_qwEgtTW1CNtpfl7ZeIb9MUmsWHIBp",
    model="solar-pro2"
)

# -----------------------------
# 2) Colab RAG 시스템 정의
# -----------------------------
def colab_rag_system(file_path):
    # 1. 문서 로드
    loader = TextLoader(file_path)
    documents = loader.load()
    print(f"문서 로딩 완료: {len(documents)} 개의 문서")

    # 2. 문서 분할
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=50,
        length_function=len
    )
    texts = text_splitter.split_documents(documents)
    print(f"문서 분할 완료: {len(texts)} 개의 청크")

    # 3. 임베딩 + 벡터저장소
    print("임베딩 모델 로딩 중...")
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    print("벡터 저장소 구축 중...")
    vectorstore = FAISS.from_documents(texts, embeddings)

    # 4. 검색기 생성
    retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

    # 5. 프롬프트 템플릿
    prompt_template = """
    다음 정보를 바탕으로 질문에 답해주세요.
    만약 관련 내용이 없다면 "관련 내용을 찾을 수 없습니다."라고 답해주세요.

    {context}

    질문: {question}
    답변:
    """
    PROMPT = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )

    # 6. RAG 파이프라인 구축
    print("RAG 파이프라인 구축 중...")
    qa_chain = RetrievalQA.from_chain_type(
        llm=chat,                    # 여기서 Solar-Pro2 사용
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
        chain_type_kwargs={"prompt": PROMPT}
    )

    print("RAG 시스템 준비 완료!")

    # 7. 대화형 질의
    while True:
        query = input("\n질문을 입력하세요 (종료하려면 'q' 입력): ")
        if query.lower() == 'q':
            break

        result = qa_chain.invoke({"query": query})

        print("\n답변:", result["result"])


    # 8. 벡터 저장소 저장
    with tempfile.TemporaryDirectory() as temp_dir:
        index_path = os.path.join(temp_dir, "faiss_index")
        vectorstore.save_local(index_path)
        print(f"\n인덱스를 '{index_path}'에 저장했습니다.")

if __name__ == "__main__":
    file_path = upload_file()
    colab_rag_system(file_path)

문서 파일을 업로드해주세요.


Saving sample-document.md to sample-document (3).md
업로드 완료: sample-document (3).md
문서 로딩 완료: 1 개의 문서
문서 분할 완료: 2 개의 청크
임베딩 모델 로딩 중...


/tmp/ipython-input-1443279852.py:48: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

벡터 저장소 구축 중...
RAG 파이프라인 구축 중...
RAG 시스템 준비 완료!

질문을 입력하세요 (종료하려면 'q' 입력): 인공지능의 역사에서 튜링 테스트란 무엇인가요?

답변: 튜링 테스트는 앨런 튜링이 1950년 논문 "Computing Machinery and Intelligence"에서 제안한 개념으로, 기계가 인간과 구별되지 않는 지능적인 행동을 보일 수 있는지를 평가하는 방법입니다. 이 테스트는 평가자(인간)가 자연어로 대화를 나눈 후, 상대방의 정체성(인간 또는 기계)을 판단하도록 구성됩니다. 평가자가 인간과 기계를 유의미하게 구분하지 못하면 해당 기계는 테스트를 통과한 것으로 간주됩니다. 이는 인공지능의 초기 이론적 기반을 마련한 중요한 이정표입니다.  

(참고: 제공된 정보 중 "# 인공지능의 역사와 발전" 섹션의 "초기 인공지능 (1940-1950년대)" 부분 참조)

질문을 입력하세요 (종료하려면 'q' 입력): 딥러닝 혁명은 언제 시작되었나요?

답변: 딥러닝 혁명은 **2010년대**에 시작되었습니다. 특히 **2012년 ImageNet 대회**에서 **알렉스넷(AlexNet)**이 혁신적인 성과를 보이며 딥러닝의 잠재력이 입증되었습니다. 이는 컴퓨팅 파워의 증가, 빅데이터의 확산, 알고리즘 개선 등이 결합된 결과로, 컴퓨터 비전, 음성 인식, 자연어 처리 등 다양한 분야에서 발전을 이끌었습니다.  

따라서 명확한 시작 시점은 **2012년**으로 볼 수 있으나, 넓은 의미에서는 **2010년대 전체**를 딥러닝 혁명의 시기로 지칭합니다.

질문을 입력하세요 (종료하려면 'q' 입력): 데이터 분석에 대해 설명하세요

답변: 제공된 자료에서는 데이터 분석에 대한 구체적인 언급이 없습니다. 따라서 "관련 내용을 찾을 수 없습니다."라고 답변드립니다.  

다만, 주어진 내용에서 **기계학습(1990-2000년대)**과 **딥러닝 혁명(2010년대-현재)**을 통해 데이터 기반 패턴 학습 및 예측이 AI 발전의 핵심 요소로 강